[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/trainer/blob/main/crawler.ipynb)

In [ ]:
import numpy as np
import requests

def fetch_image_urls_from_subreddit(subreddit, num_links, after):
    url = f"https://www.reddit.com/r/{subreddit}/new/.json?limit={num_links}&after={after}"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code == 200:
        data = response.json()
        return [post['data']['url'] for post in data['data']['children']], data['data']['after']
    else:
        print(Fore.RED + f"Error fetching image URLs: {response.status_code} - {response.text}")
        return []

items = []
after = ""
sub_reddit = "trueratecelebrities"

for i in range(10):
  new_images, after = fetch_image_urls_from_subreddit(sub_reddit, 100, after)
  items.extend(new_images)
  print(after)

!mkdir /content/images
images = []

for item in items:
    headers={'user-agent': 'Mozilla/5.0'}
    try:
      r=requests.get(item, headers=headers)
      name = item.split('/')[-1]
      with open(f"/content/images/{name}", 'wb') as f:
        f.write(r.content)
        images.append(f"/content/images/{name}")
    except Exception as e:
      pass

print(len(images))

In [ ]:
%cd /content
!pip install https://github.com/camenduru/insightface-colab/releases/download/v0.1/mmcv_full-1.4.0-cp310-cp310-linux_x86_64.whl
!pip install gradio insightface==0.6.2 terminaltables==3.1.0 mmpycocotools==12.0.3
!git clone -b dev --recurse-submodules https://github.com/camenduru/insightface-SCRFD-hf
%cd /content/insightface-SCRFD-hf
!sed -i '23,26d' /content/insightface-SCRFD-hf/insightface/detection/scrfd/mmdet/__init__.py
!wget https://huggingface.co/public-data/insightface/resolve/main/models/scrfd_34g/model.pth -O /content/model.pth

In [ ]:
%cd /content/insightface-SCRFD-hf/insightface/detection/scrfd

import torch.nn as nn
import cv2
from PIL import Image
import numpy as np
import math
import os
import torch

from mmdet.apis import inference_detector, init_detector, show_result_pyplot
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = init_detector("/content/insightface-SCRFD-hf/insightface/detection/scrfd/configs/scrfd/scrfd_34g.py", "/content/model.pth", 'cpu')

def detect(image: np.ndarray) -> np.ndarray:
    face_score_threshold = 0.5
    
    image = image[:, :, ::-1]
    preds = inference_detector(model, image)
    boxes = preds[0]

    res = image.copy()
    for box in boxes:
        box, score = box[:4], box[4]
        if score < face_score_threshold:
            continue
        box = np.round(box).astype(int)
        line_width = max(2, int(3 * (box[2:] - box[:2]).max() / 256))
        # cv2.rectangle(res, tuple(box[:2]), tuple(box[2:]), (0, 255, 0), line_width)

        center_x = (box[0] + box[2]) // 2
        center_y = (box[1] + box[3]) // 2
        center = (center_x, center_y)
        # cv2.circle(res, center, 5, (0, 0, 255), -1)

        length = math.sqrt(box[2]**2 + box[3]**2)
        x = int(center_x - int(length//2))
        y = int(center_y - int(length//2))

        if x < 0:
            x = 0
        if y < 0:
            y = 0
        if x + int(length) > res.shape[1]:
            x = res.shape[1] - int(length)
        if y + int(length) > res.shape[0]:
            y = res.shape[0] - int(length)

        width, height = int(length), int(length)
        x = center_x - (width // 2)
        y = center_y - (height // 2)

        if x < 0:
            x = 0
        if y < 0:
            y = 0
        if x + width > res.shape[1]:
            x = res.shape[1] - width
        if y + height > res.shape[0]:
            y = res.shape[0] - height

        res = res[y:y+height, x:x+width]
        res = cv2.resize(res, (512, 512))

    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    return res

!mkdir /content/cropped
directory = '/content/images'
image_paths = []
for root, _, files in os.walk(directory):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
            image_path = os.path.join(root, file)
            image_paths.append(image_path)

for image_path in image_paths:
    try:
      image_array = np.array(Image.open(image_path))
      image = Image.fromarray(detect(image_array))
      image_name = os.path.basename(image_path)
      image.save(f'/content/cropped/{image_name}')
    except Exception as e:
      print(f"{image_path}: {str(e)}")

In [ ]:
%cd /content

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y install -qq aria2
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 diffusers==0.19.0 datasets==2.14.0 gradio==3.38.0 wandb==0.15.7 transformers==4.26.0 accelerate==0.16.0 bitsandbytes==0.41.0 -U

!git clone https://github.com/camenduru/trainer

diffusers_version = "v0.19.0"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://raw.githubusercontent.com/huggingface/diffusers/{diffusers_version}/scripts/convert_diffusers_to_original_stable_diffusion.py -d /content/trainer/diffusers/dreambooth -o convert_diffusers_to_original_stable_diffusion.py
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://raw.githubusercontent.com/huggingface/diffusers/{diffusers_version}/examples/dreambooth/train_dreambooth.py -d /content/trainer/diffusers/dreambooth -o train_dreambooth.py
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://raw.githubusercontent.com/huggingface/diffusers/{diffusers_version}/examples/dreambooth/train_dreambooth_lora.py -d /content/trainer/diffusers/lora -o train_dreambooth_lora.py

BaseModelUrl = "https://huggingface.co/uf/cyberrealistic_v3.2"
BaseModelDir = "/content/model"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/model_index.json -d {BaseModelDir} -o model_index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/vae/diffusion_pytorch_model.bin -d {BaseModelDir}/vae -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/vae/config.json -d {BaseModelDir}/vae -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/unet/diffusion_pytorch_model.bin -d {BaseModelDir}/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/unet/config.json -d {BaseModelDir}/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/vocab.json -d {BaseModelDir}/tokenizer -o vocab.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/tokenizer_config.json -d {BaseModelDir}/tokenizer -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/special_tokens_map.json -d {BaseModelDir}/tokenizer -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/merges.txt -d {BaseModelDir}/tokenizer -o merges.txt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/text_encoder/pytorch_model.bin -d {BaseModelDir}/text_encoder -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/text_encoder/config.json -d {BaseModelDir}/text_encoder -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/scheduler/scheduler_config.json -d {BaseModelDir}/scheduler -o scheduler_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/safety_checker/pytorch_model.bin -d {BaseModelDir}/safety_checker -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/safety_checker/config.json -d {BaseModelDir}/safety_checker -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/feature_extractor/preprocessor_config.json -d {BaseModelDir}/feature_extractor -o preprocessor_config.json

%cd /content/trainer
!python realistic.py